<a href="https://colab.research.google.com/github/CamiloVga/Codes/blob/main/SyntheticData_FineTune_Classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Generar Datos sintéticos

!pip install openai==0.28
import openai
import pandas as pd

# Reemplazar 'SU_CLAVE_DE_API' con su clave de API de OpenAI
openai.api_key = 'API KEY'

# Variables para la generación de texto
num_ejemplos = 100

# Generar la base de datos con la API de OpenAI
prompt = "Escribe un trino sobre una reseña de un restaurante. El trino puede ser aleatoriamente bueno o malo sobre el restaurante"
base_datos = []
for _ in range(num_ejemplos):
    mensajes = [{"role": "user", "content": prompt}]
    respuesta = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=mensajes,
        max_tokens=50,
        temperature=1.5,
        n=1,
    )
    texto = respuesta["choices"][0]["message"]
    contenido_mensaje = texto["content"]
    base_datos.append(contenido_mensaje)

# Variable para la instrucción de etiquetado
instruccion_etiquetado = "Clasifica el texto según las siguientes categorías de tono: Bueno, Malo. Solo responde con base a estas categorías, sin texto adicional ni puntos o espacios. Y solo se puede elegir una categoria"

# Crear la columna 'label' utilizando la API de OpenAI para el etiquetado
etiquetas = []
for texto in base_datos:
    mensajes = [
        {"role": "system", "content": instruccion_etiquetado},
        {"role": "user", "content": texto}
    ]
    respuesta = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=mensajes,
        max_tokens=150,
        temperature=0.5,
        n=1,
        stop=None,
    )["choices"][0]["message"]["content"]
    etiquetas.append(respuesta)

# Combinar el texto y las etiquetas en un DataFrame
df = pd.DataFrame({"text": base_datos, "label": etiquetas})

# Guardar el DataFrame en un archivo CSV
df.to_csv("train.csv", index=False)

# Mostrar el DataFrame resultante
df


,text,label
0,"""¡Qué noche tan maravillosa en ese nuevo resta...",Bueno
1,"""¡Increíble experiencia en este restaurante! L...",Bueno
2,"""La comida mediocre, el servicio lento y los p...",Malo
3,"""La comida en este restaurante es una explosio...",Bueno
4,"""Increíble experiencia culinaria en el restaur...",Bueno
...,...,...
95,"""No puedo creer lo deliciosa que estaba la com...",Bueno
96,"""¡Increíble experiencia en este restaurante! L...",Bueno
97,"""La reseña del restaurante menciona platos del...",Bueno
98,"""La comida de este restaurante es una explosió...",Bueno


In [ ]:
#Convertir la base generada en de labels númericos

# Mapear las clases a números
etiquetas_numericas = {"Bueno": 0, "Malo": 1}
etiquetas_numericas_convertidas = [etiquetas_numericas[etiqueta] for etiqueta in etiquetas]

# Combinar el texto y las etiquetas numéricas en un nuevo DataFrame
nuevo_df = pd.DataFrame({"text": base_datos, "label": etiquetas_numericas_convertidas})

# Guardar el nuevo DataFrame en un archivo CSV
nuevo_df.to_csv("train_numerico.csv", index=False)

# Mostrar el nuevo DataFrame resultante
nuevo_df

,text,label
0,"""¡Qué noche tan maravillosa en ese nuevo resta...",0
1,"""¡Increíble experiencia en este restaurante! L...",0
2,"""La comida mediocre, el servicio lento y los p...",1
3,"""La comida en este restaurante es una explosio...",0
4,"""Increíble experiencia culinaria en el restaur...",0
...,...,...
95,"""No puedo creer lo deliciosa que estaba la com...",0
96,"""¡Increíble experiencia en este restaurante! L...",0
97,"""La reseña del restaurante menciona platos del...",0
98,"""La comida de este restaurante es una explosió...",0


In [ ]:
# Transformers installation
!pip install transformers datasets  # Instalar paquetes transformers y datasets
!pip install transformers[torch]  # Instalar transformers con soporte para PyTorch
!pip install accelerate  # Instalar accelerate
!pip install evaluate  # Instalar evaluate

import pandas as pd
from datasets import load_dataset, Dataset  # Importar Dataset de datasets
from transformers import AutoTokenizer  # Importar AutoTokenizer de transformers
from transformers import AutoModelForSequenceClassification  # Importar AutoModelForSequenceClassification de transformers
from transformers import TrainingArguments, Trainer  # Importar TrainingArguments y Trainer de transformers
import numpy as np  # Importar numpy
import evaluate  # Importar evaluate

# Cargar el conjunto de datos desde el archivo CSV
dataset = pd.read_csv("train_numerico.csv")

# Dividir el conjunto de datos en conjuntos de entrenamiento y evaluación
from sklearn.model_selection import train_test_split

# Asumiendo que tienes columnas llamadas "text" y "label"
train_texts, eval_texts, train_labels, eval_labels = train_test_split(dataset["text"], dataset["label"], test_size=0.2, random_state=42)

# Crear diccionarios para los conjuntos de entrenamiento y evaluación
train_dataset = {"text": train_texts, "label": train_labels}
eval_dataset = {"text": eval_texts, "label": eval_labels}

# Convertir diccionarios a objetos Dataset
train_dataset = Dataset.from_dict(train_dataset)
eval_dataset = Dataset.from_dict(eval_dataset)

tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")  # Cargar tokenizador BERT

def tokenize_function(examples):  # Definir función para tokenizar
    return tokenizer(examples["text"], truncation=True, padding="max_length", max_length=512)

tokenized_train_dataset = train_dataset.map(tokenize_function, batched=True)  # Aplicar tokenización al conjunto de entrenamiento
tokenized_eval_dataset = eval_dataset.map(tokenize_function, batched=True)  # Aplicar tokenización al conjunto de evaluación

# Agregar las etiquetas al conjunto de datos tokenizado
tokenized_train_dataset = tokenized_train_dataset.add_column("labels", train_labels)
tokenized_eval_dataset = tokenized_eval_dataset.add_column("labels", eval_labels)

model = AutoModelForSequenceClassification.from_pretrained("bert-base-cased", num_labels=len(dataset["label"].unique()))  # Cargar modelo BERT para clasificación de secuencias

training_args = TrainingArguments(output_dir="restaurants-reviews", evaluation_strategy="epoch")  # Definir hiperparámetros de entrenamiento con evaluación por época

metric = evaluate.load("accuracy")  # Cargar métrica de precisión

def compute_metrics(eval_pred):  # Definir función para calcular métricas
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_eval_dataset,
    compute_metrics=compute_metrics,
)  # Crear objeto Trainer

trainer.train()  # Entrenar el modelo

Map:   0%|          | 0/80 [00:00<?, ? examples/s]

Map:   0%|          | 0/20 [00:00<?, ? examples/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.435200,0.850000
2,No log,0.408831,0.800000
3,No log,0.393477,0.800000


TrainOutput(global_step=30, training_loss=0.48072579701741536, metrics={'train_runtime': 23.6452, 'train_samples_per_second': 10.15, 'train_steps_per_second': 1.269, 'total_flos': 63146653286400.0, 'train_loss': 0.48072579701741536, 'epoch': 3.0})

In [ ]:
# Run text generation pipeline with our next model
from transformers import pipeline  # Importar pipeline de transformers
prompt = "food is very very good, and perfect service"
pipe = pipeline(task="text-classification", model=model, tokenizer=tokenizer)  # Crear pipeline de clasificación de texto
result = pipe(f"<s>[INST] {prompt} [/INST] \n\n")  # Realizar inferencia con texto de ejemplo
print(result)  # Imprimir resultado



[{'label': 'LABEL_0', 'score': 0.787483274936676}]


In [ ]:
#Mejorar la respuesta del modelo

# Diccionario para convertir etiquetas numéricas a etiquetas originales
etiquetas_originales = {"LABEL_0": "Bueno", "LABEL_1": "Malo"}

def convertir_predicciones(predicciones):
    etiquetas_predichas = []
    for prediccion in predicciones:
        etiqueta_predicha = None
        for clave, valor in etiquetas_originales.items():
            if clave == prediccion:
                etiqueta_predicha = valor
                break
        if etiqueta_predicha is None:
            etiqueta_predicha = prediccion
        etiquetas_predichas.append(etiqueta_predicha)
    return etiquetas_predichas

# Ejemplo de inferencia y conversión de predicciones
prompt = "food is very very nice"
pipe = pipeline(task="text-classification", model=model, tokenizer=tokenizer)
result = pipe(prompt)[0]
prediccion_numerica = result['label']
etiqueta_predicha = convertir_predicciones([prediccion_numerica])[0]
print(f"Predicción: {etiqueta_predicha}")


Predicción: Bueno


In [ ]:
#Login en huggingFace

from huggingface_hub import notebook_login  # Importar notebook_login de huggingface_hub
notebook_login()  # Iniciar sesión en Hugging Face Hub

In [ ]:
"""# Publicar en Hugging Face"""

trainer.push_to_hub()  # Publicar modelo en Hugging Face Hub

model.safetensors:   0%|          | 0.00/433M [00:00<?, ?B/s]

training_args.bin:   0%|          | 0.00/4.92k [00:00<?, ?B/s]

Upload 3 LFS files:   0%|          | 0/3 [00:00<?, ?it/s]

events.out.tfevents.1713445759.0f73bd35b1e2.642.2:   0%|          | 0.00/5.86k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/CamiloVega/restaurants-reviews/commit/d20d4aa17b15ce2d6f04e36409558432a6842fe4', commit_message='End of training', commit_description='', oid='d20d4aa17b15ce2d6f04e36409558432a6842fe4', pr_url=None, pr_revision=None, pr_num=None)